# Adversarial Images for MNIST Classifier
Create adversarial images to fool a MNIST classifier in TensorFlow.

## Introduction

Convolutional Neural Networks acheive near "human level performance" for classifying images. However, they can also be [fooled easily](https://arxiv.org/abs/1312.6199) by introducing a small amount of noise into the image, which is often imperceptible to the human eye. This type of image is sometimes called an adversarial example.

In this short notebook, I will train a MNIST classifier using the [Deep MNIST for Exports] tensorflow tutorial and introduce noise into pictures of '2's that the model will misclassify as '6's.

### Imports

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [4]:
tf.__version__

'0.12.1'

### Load MNIST data

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Start Tensorflow Interactive Session

In [6]:
sess = tf.InteractiveSession()

## Train the Convolutional Neural Network

### Placeholders

Start by creating placeholders for the input images (x) and output classes (y), which we will feed into our model in batches.

In [10]:
x = tf.placeholder(tf.float32, shape=[None, 28*28], name='x')
y_ = tf.placeholder(tf.float32, shape=[None, 1], name='y')

### Weight Initialization

In [13]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

### Convolution and Pooling

Our convolutions use a stride of one, are zero padded, and use max pooling over 2x2 blocks.

In [14]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### First Convolutional Layer

Our first layer will have a convolution followed by max pooling. 'The convolutional will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.'

In [15]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

To apply the layer, we reshape x.

In [16]:
x_image = tf.reshape(x, [-1,28,28,1])

And apply the convolution to the x_image using W_conv1, add the bias, feed through the RELU function, and apply max pooling.

In [17]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Second Convolutional Layer

The second layer will have 64 features for each 5x5 patch.

In [20]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Fully Connected Layer

The fully connected layer takes the $7*7*64$ input, multiples it by weight vector $(7*7*64, 1024)$, adds a bias, and applies a RELU transformation.

In [22]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Apply Dropout

Applying dropout during training introduces regularization by not allowing the model to rely too heavily on certain activations.

In [24]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer

The final layer takes the output of the fully connected layer $(1024, 1)$ and outputs a $(10,1)$ matrix with each element corresponding to a image class.

In [25]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### Train and evaluate the Model
We will use an adam optimizer (momentum and learning rate decay). 

In [26]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Initialize Variables

In [29]:
sess.run(tf.global_variables_initializer())

### Run the Model

Start with 2,000 iterations of SGD.

In [ ]:
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.12
step 100, training accuracy 0.82
